In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox
import qrcode
from PIL import Image, ImageTk
import cv2
from pyzbar.pyzbar import decode
import numpy as np
import threading
import os
import webbrowser

# ----------------- Functions -----------------

def go_home():
    for widget in root.winfo_children():
        widget.destroy()
    show_main_screen()

# --- Generate QR ---
def open_generate_qr():
    for widget in root.winfo_children():
        widget.destroy()

    tk.Label(root, text="Generate QR Code", font=("Segoe UI", 20, "bold"), fg="white", bg="#1c1c1c").pack(pady=20)

    entry = tk.Entry(root, font=("Segoe UI", 12), width=40)
    entry.pack(pady=10)

    qr_label = tk.Label(root, bg="#1c1c1c")
    qr_label.pack(pady=20)

    selected_file_path = tk.StringVar()

    def choose_image():
        file_path = filedialog.askopenfilename(
            title="Select Image",
            filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")]
        )
        if file_path:
            selected_file_path.set(file_path)
            entry.delete(0, tk.END)
            entry.insert(0, file_path)
            messagebox.showinfo("Image Selected", f"Image path added:\n{file_path}")

    def generate():
        data = entry.get().strip()
        if not data:
            messagebox.showwarning("Warning", "Please enter text or select an image.")
            return

        qr = qrcode.make(data)
        qr.save("generated_qr.png")

        img = Image.open("generated_qr.png").resize((200, 200))
        img_tk = ImageTk.PhotoImage(img)
        qr_label.config(image=img_tk)
        qr_label.image = img_tk
        messagebox.showinfo("Success", "✅ QR code saved as 'generated_qr.png'")

    tk.Button(root, text="📁 Choose Image", font=("Segoe UI", 12, "bold"),
              command=choose_image, bg="#0091ea", fg="white", width=15, height=1).pack(pady=5)

    tk.Button(root, text="Generate QR", font=("Segoe UI", 12, "bold"),
              command=generate, bg="#00bfa5", fg="white", width=15, height=1).pack(pady=5)

    tk.Button(root, text="⬅ Back", font=("Segoe UI", 12, "bold"),
              command=go_home, bg="#ff9100", fg="white", width=15, height=1).pack(pady=10)

# --- Scan QR ---
def open_scan_qr():
    for widget in root.winfo_children():
        widget.destroy()

    tk.Label(root, text="Scan QR Code", font=("Segoe UI", 20, "bold"), fg="white", bg="#1c1c1c").pack(pady=20)

    def scan_from_image():
        file_path = filedialog.askopenfilename(
            filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")]
        )
        if not file_path:
            return
        decoded = decode(Image.open(file_path))
        if decoded:
            data = decoded[0].data.decode("utf-8")
            handle_qr_data(data)
        else:
            messagebox.showerror("Error", "❌ No QR code found in image.")

    def scan_from_camera():
        def scan():
            cap = cv2.VideoCapture(0)
            found = None
            messagebox.showinfo("Camera", "📷 Scanning... Press 'Q' to quit.")

            while True:
                success, frame = cap.read()
                if not success:
                    break

                for barcode in decode(frame):
                    data = barcode.data.decode("utf-8")
                    found = data
                    pts = np.array([point for point in barcode.polygon], np.int32)
                    pts = pts.reshape((-1, 1, 2))
                    cv2.polylines(frame, [pts], True, (0, 255, 0), 3)
                    cv2.putText(frame, data, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                                1, (255, 0, 0), 2)

                cv2.imshow("QR Scanner", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            cap.release()
            cv2.destroyAllWindows()

            if found:
                handle_qr_data(found)
            else:
                messagebox.showinfo("Result", "❌ No QR code detected.")

        threading.Thread(target=scan).start()

    tk.Button(root, text="📁 Scan from Image", font=("Segoe UI", 12, "bold"),
              command=scan_from_image, bg="#0091ea", fg="white", width=20, height=1).pack(pady=10)
    tk.Button(root, text="📷 Scan with Camera", font=("Segoe UI", 12, "bold"),
              command=scan_from_camera, bg="#00c853", fg="white", width=20, height=1).pack(pady=10)
    tk.Button(root, text="⬅ Back", font=("Segoe UI", 12, "bold"),
              command=go_home, bg="#ff9100", fg="white", width=20, height=1).pack(pady=20)

# --- Handle QR Result ---
def handle_qr_data(data):
    # If it's a file path and exists
    if os.path.exists(data) and any(data.lower().endswith(ext) for ext in [".jpg", ".png", ".jpeg", ".bmp"]):
        messagebox.showinfo("QR Data", f"🖼 Image path detected:\n{data}")
        os.startfile(data)  # Opens image locally
    elif data.startswith("http"):
        messagebox.showinfo("QR Data", f"🌐 URL detected:\n{data}")
        webbrowser.open(data)
    else:
        messagebox.showinfo("QR Data", f"📝 Text: {data}")

# --- Home Screen ---
def show_main_screen():
    tk.Label(root, text="🔲 QR Toolkit", font=("Segoe UI", 22, "bold"), fg="white", bg="#1c1c1c").pack(pady=40)

    tk.Button(root, text="➕ Generate QR", font=("Segoe UI", 14, "bold"),
              command=open_generate_qr, bg="#00bfa5", fg="white", width=20, height=2).pack(pady=10)
    tk.Button(root, text="🔍 Scan QR", font=("Segoe UI", 14, "bold"),
              command=open_scan_qr, bg="#0091ea", fg="white", width=20, height=2).pack(pady=10)

    tk.Label(root, text="Runs fully offline 💻", font=("Segoe UI", 10),
             fg="gray", bg="#1c1c1c").pack(side="bottom", pady=20)

# ----------------- Main Window -----------------
root = tk.Tk()
root.title("QR Toolkit")
root.geometry("450x550")
root.config(bg="#1c1c1c")

show_main_screen()
root.mainloop()